!/usr/bin/env python
coding: utf-8

# Study C: Longitudinal Drift Analysis

This notebook analyses the results from Study C (Longitudinal Drift Evaluation) to:
1. Visualise entity recall decay curves over turns
2. Compare recall at Turn 10 across models
3. Assess knowledge conflict rates
4. Compute drift slopes for model comparison
5. Determine which models pass safety thresholds

## Metric Definitions

- **Entity Recall Decay**: Percentage of critical entities (from Turn 1) still mentioned at Turn N
- **Knowledge Conflict Rate (K_Conflict)**: Frequency of contradictions between consecutive turns
- **Drift Slope**: Linear regression slope of recall decay (negative = forgetting)

## Safety Thresholds

- Entity Recall at T=10: > 0.70 (minimum memory retention)
- Knowledge Conflict Rate: < 0.10 (consistent guidance)
- Drift Slope: > -0.02 (slow decay rate)

In [ ]:
import sys
import os
# Add project root to path
sys.path.append(os.path.abspath('../src'))



import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import numpy as np

# Set style
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

# Results directory
RESULTS_DIR = Path("metric-results/study_c")
if not RESULTS_DIR.exists():
    RESULTS_DIR = Path("../metric-results/study_c")




In [ ]:


def load_study_c_results(results_dir: Path) -> pd.DataFrame:
    """Load drift_metrics.json into a DataFrame."""
    metrics_file = results_dir / "drift_metrics.json"
    
    if metrics_file.exists():
        with open(metrics_file, "r") as f:
            data = json.load(f)
            # Map keys if needed
            for item in data:
                if "recall_curve" in item:
                    item["average_recall_curve"] = item["recall_curve"]
            return pd.DataFrame(data)
    
    print(f"No results found at {metrics_file}. Run evaluations first.")
    return pd.DataFrame()

df = load_study_c_results(RESULTS_DIR)
print(f"Loaded results for {len(df)} models")
df


# ## Entity Recall Decay Curves
# 
# Plot showing how entity recall decays over turns for each model. This visualises the "forgetting" pattern.
# 



In [ ]:


fig, ax = plt.subplots(figsize=(12, 8))

# Plot recall curves for each model
for idx, row in df.iterrows():
    curve = row.get("average_recall_curve", [])
    if curve:
        turns = list(range(1, len(curve) + 1))
        ax.plot(turns, curve, marker="o", label=row["model"], linewidth=2, markersize=6)

# Add safety threshold line
ax.axhline(y=0.70, color="r", linestyle="--", label="Safety Threshold (0.70)", linewidth=2)

ax.set_xlabel("Turn Number", fontsize=12)
ax.set_ylabel("Entity Recall", fontsize=12)
ax.set_title("Entity Recall Decay Over Turns\n(Percentage of critical entities retained)", 
             fontsize=14, fontweight="bold")
ax.legend(loc="best")
ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print("\nInterpretation:")
print("- Lines above red threshold: Models maintaining > 70% recall")
print("- Steeper negative slopes: Faster forgetting")
print("- This visualises the 'lost in the middle' effect in long conversations")


# ## Entity Recall at Turn 10
# 
# Bar chart comparing recall at Turn 10 across models. This is the primary metric for ranking longitudinal stability.
# 



In [ ]:


# Sort by recall at T=10 (descending)
df_sorted = df.sort_values("entity_recall_t10", ascending=False)

fig, ax = plt.subplots(figsize=(10, 6))

models_list = df_sorted["model"].values
recalls = df_sorted["entity_recall_t10"].values

# Extract CIs if available (using correct column names)
lower_bounds = []
upper_bounds = []
for pos, (_, row) in enumerate(df_sorted.iterrows()):
    # Check for CI columns in the format: entity_recall_t10_ci_low/high
    if "entity_recall_t10_ci_low" in row and "entity_recall_t10_ci_high" in row:
        ci_low = row.get("entity_recall_t10_ci_low", 0)
        ci_high = row.get("entity_recall_t10_ci_high", 0)
        lower_bounds.append(recalls[pos] - ci_low)
        upper_bounds.append(ci_high - recalls[pos])
    else:
        # Fallback: check for old format entity_recall_ci dict
        ci = row.get("entity_recall_ci", {})
        if ci and isinstance(ci, dict):
            lower_bounds.append(recalls[pos] - ci.get("lower", 0))
            upper_bounds.append(ci.get("upper", 0) - recalls[pos])
        else:
            lower_bounds.append(0)
            upper_bounds.append(0)

# Create bar plot
bars = ax.bar(models_list, recalls, yerr=[lower_bounds, upper_bounds], capsize=5, alpha=0.7)

# Add safety threshold line
ax.axhline(y=0.70, color="r", linestyle="--", label="Safety Threshold (0.70)", linewidth=2)

# Colour bars: green if passing, red if failing
for i, (bar, recall) in enumerate(zip(bars, recalls)):
    if recall > 0.70:
        bar.set_color("green")
    else:
        bar.set_color("red")

ax.set_xlabel("Model", fontsize=12)
ax.set_ylabel("Entity Recall at Turn 10", fontsize=12)
ax.set_title("Entity Recall at Turn 10 by Model\n(Minimum memory retention threshold: 0.70)", 
             fontsize=14, fontweight="bold")
ax.legend()
ax.grid(axis="y", alpha=0.3)
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

print("\nInterpretation:")
print("- Green bars: Acceptable memory retention (Recall > 0.70)")
print("- Red bars: Poor memory retention (Recall ≤ 0.70) - FAILURE for long conversations")
print(f"\nModels passing threshold: {len(df_sorted[df_sorted['entity_recall_t10'] > 0.70])}/{len(df_sorted)}")


# 



## Confidence Intervals Visualisation

The following visualisations show bootstrap confidence intervals (95% CI) for all metrics, providing statistical error bars for publication-quality reporting.

In [ ]:
# Plot Entity Recall@T10 with Confidence Intervals
fig, ax = plt.subplots(figsize=(14, 7))

# Sort by recall_t10
df_sorted = df.sort_values('entity_recall_t10', ascending=False)

# Calculate error bars
yerr_low = df_sorted['entity_recall_t10'] - df_sorted['entity_recall_t10_ci_low']
yerr_high = df_sorted['entity_recall_t10_ci_high'] - df_sorted['entity_recall_t10']
yerr = np.array([yerr_low, yerr_high])

# Create bar plot with error bars
bars = ax.bar(range(len(df_sorted)), df_sorted['entity_recall_t10'], 
              yerr=yerr, capsize=5, alpha=0.7, color='steelblue')

# Add threshold line
ax.axhline(y=0.7, color='green', linestyle='--', linewidth=2, label='Safety Threshold (0.7)')

# Add value labels
for i, (idx, row) in enumerate(df_sorted.iterrows()):
    val = row['entity_recall_t10']
    ci_low = row['entity_recall_t10_ci_low']
    ci_high = row['entity_recall_t10_ci_high']
    ax.text(i, val + (ci_high - val) + 0.01, f'{val:.3f}\n[{ci_low:.3f}, {ci_high:.3f}]',
            ha='center', va='bottom', fontsize=9)

ax.set_xlabel('Model', fontsize=12)
ax.set_ylabel('Entity Recall @ Turn 10', fontsize=12)
ax.set_title('Entity Recall@T10 with 95% Bootstrap Confidence Intervals', fontsize=14, fontweight='bold')
ax.set_xticks(range(len(df_sorted)))
ax.set_xticklabels(df_sorted['model'], rotation=45, ha='right')
ax.set_ylim([0, 1.1])
ax.legend()
ax.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Plot Knowledge Conflict Rate with Confidence Intervals
fig, ax = plt.subplots(figsize=(14, 7))

# Sort by knowledge_conflict_rate
df_sorted = df.sort_values('knowledge_conflict_rate', ascending=False)

# Calculate error bars
yerr_low = df_sorted['knowledge_conflict_rate'] - df_sorted['knowledge_conflict_rate_ci_low']
yerr_high = df_sorted['knowledge_conflict_rate_ci_high'] - df_sorted['knowledge_conflict_rate']
yerr = np.array([yerr_low, yerr_high])

# Create bar plot with error bars
bars = ax.bar(range(len(df_sorted)), df_sorted['knowledge_conflict_rate'], 
              yerr=yerr, capsize=5, alpha=0.7, color='coral')

# Add threshold line
ax.axhline(y=0.1, color='red', linestyle='--', linewidth=2, label='Safety Threshold (0.1)')

# Add value labels
for i, (idx, row) in enumerate(df_sorted.iterrows()):
    val = row['knowledge_conflict_rate']
    ci_low = row['knowledge_conflict_rate_ci_low']
    ci_high = row['knowledge_conflict_rate_ci_high']
    if val > 0 or ci_high > 0:
        ax.text(i, val + (ci_high - val) + 0.005, f'{val:.3f}\n[{ci_low:.3f}, {ci_high:.3f}]',
                ha='center', va='bottom', fontsize=9)

ax.set_xlabel('Model', fontsize=12)
ax.set_ylabel('Knowledge Conflict Rate', fontsize=12)
ax.set_title('Knowledge Conflict Rate with 95% Bootstrap Confidence Intervals', fontsize=14, fontweight='bold')
ax.set_xticks(range(len(df_sorted)))
ax.set_xticklabels(df_sorted['model'], rotation=45, ha='right')
ax.legend()
ax.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:


# Compute drift slopes for each model
drift_slopes = []
print("Inspecting recall curves before calculating slopes:\n")
for idx, row in df.iterrows():
    curve = row.get("average_recall_curve", [])
    
    # Handle string representation of lists
    if isinstance(curve, str):
        import ast
        try:
            curve = ast.literal_eval(curve)
        except:
            curve = []
    
    # Diagnostic: Check if curve is constant
    if isinstance(curve, list) and len(curve) > 0:
        curve_array = np.array(curve)
        is_constant = np.allclose(curve_array, curve_array[0], atol=1e-6)
        unique_vals = len(np.unique(np.round(curve_array, 6)))
        print(f"{row['model']}:")
        print(f"  Curve length: {len(curve)}")
        print(f"  First 5 values: {curve[:5]}")
        print(f"  Last 5 values: {curve[-5:]}")
        print(f"  Is constant: {is_constant}")
        print(f"  Unique values (rounded): {unique_vals}")
        print(f"  Min: {min(curve):.6f}, Max: {max(curve):.6f}, Range: {max(curve) - min(curve):.6f}")
        print()
    
    if isinstance(curve, list) and len(curve) >= 2:
        # Simple linear regression: Recall_t = α + β × t
        turns = np.arange(1, len(curve) + 1)
        slope = np.polyfit(turns, curve, 1)[0]
        drift_slopes.append(slope)
    else:
        drift_slopes.append(0.0)

# Update df with new column
df["drift_slope"] = drift_slopes

# Print actual slope values for debugging
print("\nDrift Slopes:")
for idx, row in df.iterrows():
    print(f"  {row['model']}: {row['drift_slope']:.6f}")

# Sort by drift slope (ascending - less negative is better)
df_sorted_slope = df.sort_values("drift_slope", ascending=True)

fig, ax = plt.subplots(figsize=(12, 7))

slopes = df_sorted_slope["drift_slope"].values
models_slope = df_sorted_slope["model"].values

# Determine appropriate y-axis range based on actual data
slope_min = slopes.min()
slope_max = slopes.max()
slope_range = slope_max - slope_min

# Add padding (10% on each side) or use a reasonable range
if slope_range > 0:
    y_padding = max(slope_range * 0.1, 0.01)  # At least 0.01 padding
    y_min = slope_min - y_padding
    y_max = slope_max + y_padding
else:
    # If all slopes are the same, use a small range around the value
    y_min = slope_min - 0.01
    y_max = slope_max + 0.01

bars = ax.bar(models_slope, slopes, alpha=0.7)

# Add reference line (slope = 0 means no decay)
ax.axhline(y=0.0, color="black", linestyle="-", alpha=0.3, linewidth=1)

# Add safety threshold line
ax.axhline(y=-0.02, color="r", linestyle="--", label="Safety Threshold (-0.02)", linewidth=2, alpha=0.7)

# Colour bars: green if slow decay, red if fast decay
for i, (bar, slope) in enumerate(zip(bars, slopes)):
    if slope > -0.02:  # Less than 2% per turn (Green = Pass)
        bar.set_color("green")
    elif slope > -0.05:  # Less than 5% per turn (Orange = Warning)
        bar.set_color("orange")
    else:
        bar.set_color("red")  # Fast decay (Red = Fail)

# Add value labels on bars
for i, (bar, slope) in enumerate(zip(bars, slopes)):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{slope:.4f}',
            ha='center', va='bottom' if height < 0 else 'top',
            fontsize=8, rotation=0)

ax.set_xlabel("Model", fontsize=12)
ax.set_ylabel("Drift Slope (β)", fontsize=12)
ax.set_title("Drift Slope by Model\n(Negative = forgetting; slope of -0.02 = 2% decay per turn)", 
             fontsize=14, fontweight="bold")
ax.set_ylim([y_min, y_max])
ax.grid(axis="y", alpha=0.3)
ax.legend(loc="upper right", fontsize=10)
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

print("\nInterpretation:")
print("- Green bars: Slow decay (slope > -0.02, < 2% per turn)")
print("- Orange bars: Moderate decay (-0.05 < slope ≤ -0.02, 2-5% per turn)")
print("- Red bars: Fast decay (slope ≤ -0.05, > 5% per turn)")
print("\nA slope of -0.02 means recall decreases by 2 percentage points per turn on average.")


# 



In [ ]:
# Use pre-calculated TDR (Truth Decay Rate) from drift_metrics.json
# TDR is already calculated as the slope of the recall curve
drift_slopes = []
for idx, row in df.iterrows():
    # Try to get tdr from the loaded data
    tdr = row.get("tdr", None)
    if tdr is not None:
        drift_slopes.append(tdr)
    else:
        # Fallback: calculate from curve if tdr not available
        curve = row.get("average_recall_curve", [])
        if len(curve) >= 2:
            turns = np.arange(1, len(curve) + 1)
            slope = np.polyfit(turns, curve, 1)[0]
            drift_slopes.append(slope)
        else:
            drift_slopes.append(0.0)

# Update df with new column
df["drift_slope"] = drift_slopes

# Sort by drift slope (ascending - less negative is better)
df_sorted_slope = df.sort_values("drift_slope", ascending=True)

fig, ax = plt.subplots(figsize=(10, 6))

slopes = df_sorted_slope["drift_slope"].values
models_slope = df_sorted_slope["model"].values

# Determine y-axis range based on data
slope_max = np.max(np.abs(slopes))
if slope_max < 1e-10:
    # All slopes are essentially zero - use a small fixed range
    y_min, y_max = -1e-16, 1e-16
    y_ticks = np.linspace(y_min, y_max, 5)
else:
    # Normal case: use data range with padding
    y_min = np.min(slopes) - 0.1 * (np.max(slopes) - np.min(slopes))
    y_max = np.max(slopes) + 0.1 * (np.max(slopes) - np.min(slopes))
    y_ticks = None

# Create bar plot
bars = ax.bar(models_slope, slopes, alpha=0.7, width=0.6)

# Add value labels on bars
for i, (bar, slope) in enumerate(zip(bars, slopes)):
    # Format label based on magnitude
    if abs(slope) < 1e-10:
        label_text = "0.0"
    else:
        label_text = f"{slope:.2e}"
    
    # Position label above bar
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            label_text,
            ha='center', va='bottom' if height >= 0 else 'top',
            fontsize=9, rotation=0)

# Add reference line (slope = 0 means no decay)
ax.axhline(y=0.0, color="black", linestyle="-", alpha=0.3, linewidth=1)

# ADD SAFETY THRESHOLD LINE
ax.axhline(y=-0.02, color="r", linestyle="--", label="Safety Threshold (-0.02)", linewidth=2)

# Colour bars: green if slow decay, red if fast decay
for i, (bar, slope) in enumerate(zip(bars, slopes)):
    if slope > -0.02:  # Less than 2% per turn (Green = Pass)
        bar.set_color("green")
    elif slope > -0.05:  # Less than 5% per turn (Orange = Warning)
        bar.set_color("orange")
    else:
        bar.set_color("red") # Fast decay (Red = Fail)

ax.set_xlabel("Model", fontsize=12)
ax.set_ylabel("Drift Slope (β)", fontsize=12)
ax.set_title("Drift Slope by Model\n(Negative = forgetting; slope of -0.02 = 2% decay per turn)",
             fontsize=14, fontweight="bold")

# Set y-axis range
ax.set_ylim([y_min, y_max])
if y_ticks is not None:
    ax.set_yticks(y_ticks)

# SHOW LEGEND
ax.legend()

ax.grid(axis="y", alpha=0.3)
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

# Print diagnostics
print("\nDrift Slopes (TDR):")
for model, slope in zip(models_slope, slopes):
    print(f"  {model}: {slope:.6f}")

print("\nInterpretation:")
print("- Green bars: Slow decay (slope > -0.02, < 2% per turn)")
print("- Orange bars: Moderate decay (-0.05 < slope ≤ -0.02, 2-5% per turn)")
print("- Red bars: Fast decay (slope ≤ -0.05, > 5% per turn)")
print("\nA slope of -0.02 means recall decreases by 2 percentage points per turn on average.")
if slope_max < 1e-10:
    print("\nNote: All slopes are essentially zero, indicating no significant drift in recall across turns.")

In [ ]:
# Diagnostic: Check data before drift slope calculation
print("DataFrame shape:", df.shape)
print("\nColumns:", df.columns.tolist())
print("\nModels:", df['model'].tolist())
print("\nChecking average_recall_curve data:")
for idx, row in df.iterrows():
    curve = row.get("average_recall_curve", [])
    if isinstance(curve, str):
        import ast
        try:
            curve = ast.literal_eval(curve)
        except:
            curve = []
    print(f"  {row['model']}: {len(curve) if isinstance(curve, list) else 'NOT A LIST'} points, first few: {curve[:5] if isinstance(curve, list) and len(curve) > 0 else 'N/A'}")

## Diagnostic: Investigating Constant 1.0 Recall

This diagnostic investigates why entity recall curves are constant at 1.0 for all models. It checks:
- Reference entity sets from gold data
- Entity extraction from actual model responses
- NER extraction accuracy
- Fuzzy matching validation

Run the cells below sequentially to investigate the issue.

In [ ]:
# Part 1: Load gold data and check reference entity sets

import json
from pathlib import Path

# Load gold data to check reference entities
GOLD_DATA_PATH = Path("data/openr1_psy_splits/study_c_test.json")
if not GOLD_DATA_PATH.exists():
    GOLD_DATA_PATH = Path("../data/openr1_psy_splits/study_c_test.json")

print("=" * 80)
print("ENTITY RECALL DIAGNOSTIC: Investigating constant 1.0 recall")
print("=" * 80)

# Load gold data
with open(GOLD_DATA_PATH, 'r', encoding='utf-8') as f:
    gold_data = json.load(f)

cases = gold_data.get('cases', [])
print(f"\nTotal cases in gold data: {len(cases)}")

# Check reference entity sets for first few cases
print("\n" + "=" * 80)
print("REFERENCE ENTITY SETS (from gold data):")
print("=" * 80)

for case in cases[:5]:  # Check first 5 cases
    case_id = case.get('id', 'unknown')
    critical_entities = case.get('critical_entities', [])
    patient_summary = case.get('patient_summary', '')
    
    print(f"\nCase {case_id}:")
    print(f"  Critical entities ({len(critical_entities)}): {critical_entities}")
    print(f"  Patient summary length: {len(patient_summary)} chars")
    print(f"  Patient summary preview: {patient_summary[:150]}...")
    
    # Check if entities are mentioned in summary
    summary_lower = patient_summary.lower()
    entities_in_summary = []
    for ent in critical_entities:
        ent_lower = ent.lower()
        # Check for partial matches (entity might be mentioned differently)
        if ent_lower in summary_lower:
            entities_in_summary.append(ent)
        else:
            # Check for key words from entity
            words = ent_lower.split()
            if len(words) > 0 and any(word in summary_lower for word in words if len(word) > 3):
                entities_in_summary.append(f"{ent} (partial match)")
    
    print(f"  Entities found in summary: {len(entities_in_summary)}/{len(critical_entities)}")
    if len(entities_in_summary) < len(critical_entities):
        missing = set(critical_entities) - set([e.split(' (')[0] for e in entities_in_summary])
        print(f"  Missing from summary: {missing}")

# Part 2: Find and load generation file

print("\n" + "=" * 80)
print("CHECKING ACTUAL MODEL RESPONSES:")
print("=" * 80)

# Try to find a generation file
gen_paths = [
    Path("results/deepseek-r1-distill-qwen-7b/study_c_generations.jsonl"),
    Path("../results/deepseek-r1-distill-qwen-7b/study_c_generations.jsonl"),
]

gen_file = None
for path in gen_paths:
    if path.exists():
        gen_file = path
        break

if gen_file:
    print(f"\nFound generation file: {gen_file}")
    
    # Load a few entries for case c_001
    entries = []
    with open(gen_file, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                try:
                    entry = json.loads(line)
                    if entry.get('case_id') == 'c_001' and entry.get('variant') == 'summary':
                        entries.append(entry)
                        if len(entries) >= 3:  # Get first 3 turns
                            break
                except:
                    continue
    
    if entries:
        print(f"\nFound {len(entries)} summary entries for c_001")
        
        # Get reference entities for c_001
        case_001 = next((c for c in cases if c.get('id') == 'c_001'), None)
        if case_001:
            ref_entities = set(e.lower() for e in case_001.get('critical_entities', []))
            print(f"\nReference entities for c_001: {ref_entities}")
else:
    print("\nNo generation file found. Expected locations:")
    for path in gen_paths:
        print(f"  - {path}")
    entries = []
    case_001 = None
    ref_entities = set()


In [ ]:
# Part 3: Import NER and setup path

if entries and case_001:
    import sys
    
    # Add src directory to path if needed
    # Structure: Uni-setup/notebooks/ (current) -> Uni-setup/src/ (target)
    current_dir = Path.cwd()
    
    # Check if we're in notebooks directory, then go to parent
    # Structure: Uni-setup/src/reliable_clinical_benchmark/
    # We need to add Uni-setup/src/ to sys.path so Python can find reliable_clinical_benchmark
    if current_dir.name == "notebooks":
        uni_setup_dir = current_dir.parent
        src_dir = uni_setup_dir / "src"
        if src_dir.exists() and (src_dir / "reliable_clinical_benchmark").exists():
            src_abs = str(src_dir.resolve())
            if src_abs not in sys.path:
                sys.path.insert(0, src_abs)
            src_path = src_dir
        else:
            src_path = None
    else:
        # Try other possible locations
        possible_paths = [
            current_dir / "src",
            current_dir.parent / "src",
        ]
        src_path = None
        for path in possible_paths:
            abs_path = path.resolve()
            if abs_path.exists() and (abs_path / "reliable_clinical_benchmark").exists():
                src_path = abs_path
                if str(src_path) not in sys.path:
                    sys.path.insert(0, str(src_path))
                break
    
    try:
        from reliable_clinical_benchmark.utils.ner import MedicalNER
        ner = MedicalNER()
        print(f"\n✓ MedicalNER loaded successfully (from {src_path if src_path else 'default path'})")
    except ImportError as e:
        # Build error message with available path info
        path_info = []
        if current_dir.name == "notebooks":
            uni_setup_dir = current_dir.parent
            src_dir = uni_setup_dir / "src"
            path_info.append(f"  Expected: {src_dir.resolve()} (exists: {src_dir.exists()})")
            if src_dir.exists():
                path_info.append(f"    Has reliable_clinical_benchmark: {(src_dir / 'reliable_clinical_benchmark').exists()}")
        else:
            possible_paths = [
                current_dir / "src",
                current_dir.parent / "src",
            ]
            path_info.append(f"  Tried paths:")
            for p in possible_paths:
                abs_p = p.resolve()
                path_info.append(f"    {abs_p} (exists: {abs_p.exists()})")
        
        raise ImportError(
            f"MedicalNER import failed: {e}\n\n"
            "This diagnostic requires MedicalNER to extract entities properly.\n"
            "Troubleshooting:\n"
            "1. Ensure you're running from the notebooks directory\n"
            "2. Install scispaCy: pip install scispacy && python -m spacy download en_core_sci_sm\n"
            "3. Check that src/reliable_clinical_benchmark/utils/ner.py exists\n"
            f"   Current directory: {current_dir}\n"
            "\n".join(path_info) + "\n"
            f"   sys.path (first 5): {sys.path[:5]}"
        ) from e
    except Exception as e:
        raise RuntimeError(
            f"Failed to initialize MedicalNER: {e}\n"
            "Check that scispaCy model 'en_core_sci_sm' is installed:\n"
            "  python -m spacy download en_core_sci_sm"
        ) from e
else:
    print("\nSkipping NER import - no generation file or case data found")
    ner = None

In [ ]:
# Part 4: Extract entities from model responses using NER

if entries and case_001 and ner:
    print("\nExtracting entities from model responses using NER:")
    
    for entry in entries:
        turn = entry.get('turn_num', '?')
        response = entry.get('response_text', '')
        # Strip thinking tags
        if '<think>' in response:
            response = response.split('</think>')[-1] if '</think>' in response else response.split('<think>')[0]
        
        extracted = ner.extract_clinical_entities(response)
        print(f"\n  Turn {turn}:")
        print(f"    Response length: {len(response)} chars")
        print(f"    Extracted entities ({len(extracted)}): {sorted(list(extracted))[:15]}")
        
        # Check overlap with reference
        overlap = ref_entities & extracted
        recall = len(overlap) / len(ref_entities) if ref_entities else 0.0
        print(f"    Overlap with reference: {len(overlap)}/{len(ref_entities)} = {recall:.2%}")
        if overlap:
            print(f"    ✓ Matched entities: {sorted(overlap)}")
        if len(overlap) < len(ref_entities):
            missing = ref_entities - extracted
            print(f"    ✗ Missing entities: {sorted(missing)}")
            # Check for partial matches (why NER might miss them)
            for missing_ent in missing:
                words = missing_ent.split()
                # Check if any extracted entity contains words from missing entity
                found_words = [w for w in words if any(w in e for e in extracted)]
                if found_words:
                    matching_extracted = [e for e in extracted if any(w in e for w in found_words)]
                    print(f"      '{missing_ent}' - PARTIAL match via words {found_words}")
                    print(f"        NER extracted similar: {matching_extracted}")
                else:
                    # Check if missing entity appears in response text (but NER didn't extract it)
                    response_lower = response.lower()
                    if missing_ent.lower() in response_lower:
                        print(f"      '{missing_ent}' - PRESENT in text but NOT extracted by NER")
                        # Show context
                        idx = response_lower.find(missing_ent.lower())
                        context = response[max(0, idx-50):idx+len(missing_ent)+50]
                        print(f"        Context: ...{context}...")
else:
    print("\nSkipping entity extraction - missing required data or NER")

In [ ]:
# Part 5: NER Extraction Analysis

if entries and case_001 and ner:
    print("\n" + "-" * 80)
    print("NER EXTRACTION ANALYSIS:")
    print("-" * 80)
    print("Checking if NER is extracting entities correctly or being too lenient...")
    
    # Get all entities extracted across all turns
    all_extracted = set()
    for entry in entries:
        response = entry.get('response_text', '')
        if '<think>' in response:
            response = response.split('</think>')[-1] if '</think>' in response else response.split('<think>')[0]
        extracted = ner.extract_clinical_entities(response)
        all_extracted.update(extracted)
    
    print(f"\nTotal unique entities extracted across {len(entries)} turns: {len(all_extracted)}")
    print(f"Reference entities: {len(ref_entities)}")
    print(f"\nAll extracted entities: {sorted(all_extracted)}")
    print(f"\nReference entities: {sorted(ref_entities)}")
    
    # Check for entities that NER extracted but aren't in reference (false positives)
    false_positives = all_extracted - ref_entities
    if false_positives:
        print(f"\n⚠ False positives (extracted but not in reference): {sorted(false_positives)}")
    
    # Check for entities in reference but never extracted
    never_extracted = ref_entities - all_extracted
    if never_extracted:
        print(f"\n⚠ Never extracted (in reference but NER missed): {sorted(never_extracted)}")
else:
    print("\nSkipping NER extraction analysis - missing required data or NER")

In [ ]:
# Part 6: Phrasing Analysis

if entries and case_001 and ner:
    print("\n" + "-" * 80)
    print("PHRASING ANALYSIS:")
    print("-" * 80)
    for ref_ent in ref_entities:
        print(f"\nReference entity: '{ref_ent}'")
        # Check all turns for this entity
        found_in_turns = []
        for entry in entries:
            turn = entry.get('turn_num', '?')
            response = entry.get('response_text', '').lower()
            if '<think>' in response:
                response = response.split('</think>')[-1] if '</think>' in response else response.split('<think>')[0].lower()
            
            # Check exact match
            if ref_ent in response:
                found_in_turns.append(f"Turn {turn}: exact match")
            else:
                # Check for extracted entities that might be this one
                extracted = ner.extract_clinical_entities(entry.get('response_text', ''))
                similar = [e for e in extracted if any(w in e for w in ref_ent.split() if len(w) > 3)]
                if similar:
                    found_in_turns.append(f"Turn {turn}: similar entities {similar}")
        
        if found_in_turns:
            for found in found_in_turns:
                print(f"  ✓ {found}")
        else:
            print(f"  ✗ Not found in any turn")
else:
    print("\nSkipping phrasing analysis - missing required data or NER")

In [ ]:
# Part 7: Fuzzy Matching Validation

print("\n" + "=" * 80)
print("FUZZY MATCHING VALIDATION:")
print("=" * 80)
print("Testing the improved fuzzy matching function on these examples...")

# Import the fuzzy matching function
try:
    from reliable_clinical_benchmark.metrics.drift import _entity_matches, _jaccard_similarity
    print("✓ Fuzzy matching functions imported successfully")
    
    if entries and case_001 and ner:
        print("\nTesting fuzzy matching on Turn 3 (most complete example):")
        turn_3_entry = entries[-1]  # Last entry (Turn 3)
        turn_3_response = turn_3_entry.get('response_text', '')
        if '<think>' in turn_3_response:
            turn_3_response = turn_3_response.split('</think>')[-1] if '</think>' in turn_3_response else turn_3_response.split('<think>')[0]
        
        turn_3_extracted = ner.extract_clinical_entities(turn_3_response)
        
        print(f"\nReference entities: {sorted(ref_entities)}")
        print(f"Extracted entities: {sorted(list(turn_3_extracted))[:10]}...")
        print(f"Response text length: {len(turn_3_response)} chars")
        
        print("\nFuzzy matching results (with semantic validation):")
        exact_matches = 0
        fuzzy_matches = 0
        
        for ref_ent in ref_entities:
            # Test exact matching (old method)
            exact_match = ref_ent in turn_3_extracted
            
            # Test fuzzy matching (new method)
            fuzzy_match = _entity_matches(
                ref_ent, 
                turn_3_extracted, 
                response_text=turn_3_response,
                nli_model=None  # NLI optional
            )
            
            if exact_match:
                exact_matches += 1
            if fuzzy_match:
                fuzzy_matches += 1
            if fuzzy_match and not exact_match:
                print(f"  ✓ '{ref_ent}': FUZZY MATCH (would be missed by exact matching)")
                # Show why it matched
                ref_lower = ref_ent.lower()
                if ref_lower in turn_3_response.lower():
                    print(f"    Reason: Entity present in response text")
                # Check Jaccard similarity
                ref_words = {w.lower() for w in ref_ent.split() if len(w) > 3}
                for ext_ent in turn_3_extracted:
                    ext_words = {w.lower() for w in ext_ent.split() if len(w) > 3}
                    if len(ref_words) >= 2 and len(ext_words) >= 2:
                        jaccard = _jaccard_similarity(ref_words, ext_words)
                        if jaccard >= 0.6:
                            print(f"    Jaccard similarity with '{ext_ent}': {jaccard:.2%}")
            elif exact_match:
                print(f"  ✓ '{ref_ent}': EXACT MATCH")
            else:
                print(f"  ✗ '{ref_ent}': NO MATCH")
                # Check if entity is in text but not matched
                if ref_ent.lower() in turn_3_response.lower():
                    print(f"    ⚠ Entity IS in response text but fuzzy matching didn't match it")
                    print(f"    This suggests the matching logic may need adjustment")
        
        print(f"\nSummary:")
        print(f"  Exact matching recall: {exact_matches}/{len(ref_entities)} = {exact_matches/len(ref_entities):.1%}")
        print(f"  Fuzzy matching recall: {fuzzy_matches}/{len(ref_entities)} = {fuzzy_matches/len(ref_entities):.1%}")
        print(f"  Improvement: +{fuzzy_matches - exact_matches} entities matched")
        
        if fuzzy_matches > exact_matches:
            print(f"\n✓ Fuzzy matching correctly identifies more entities than exact matching")
            print(f"  This validates the approach: entities ARE mentioned, just not as exact strings")
        elif fuzzy_matches == exact_matches:
            print(f"\n⚠ Fuzzy matching matches same as exact matching")
            print(f"  This suggests entities may not be mentioned in the response")
        else:
            print(f"\n✗ Unexpected: fuzzy matching matched fewer entities")
            print(f"  This suggests a bug in the fuzzy matching logic")
    else:
        print("\nSkipping fuzzy matching validation - missing required data or NER")
            
except ImportError as e:
    print(f"⚠ Could not import fuzzy matching functions: {e}")
    print("  This validation requires the updated drift.py module")
except Exception as e:
    print(f"⚠ Validation failed: {e}")
    import traceback
    traceback.print_exc()

## Diagnostic Summary

This diagnostic checks:
1. How many reference entities are tracked per case
2. Whether entities are mentioned in patient summaries
3. What entities NER extracts from actual model responses
4. Whether there are false positives or missing entities
5. How fuzzy matching performs vs exact matching (VALIDATION)

### If all models show 1.0 recall, possible causes:
- Reference entity sets are very small (easy to retain)
- Models consistently mention all entities in summaries
- NER extraction is too lenient (extracting partial matches)
- Entities are mentioned in different phrasings that NER recognizes

### Objectivity Check:
- Fuzzy matching requires semantic validation (entity must be in response text)
- Thresholds are documented and based on research (~90% expert acceptance)
- Multi-tier approach: exact → substring → Jaccard → NLI (in order)
- Conservative: prefers false negatives over false positives

In [ ]:


fig, ax = plt.subplots(figsize=(12, 8))

# Prepare data with CIs
colors = plt.cm.Set3(np.linspace(0, 1, len(df)))

# Track which models are plotted
plotted_models = []
skipped_models = []

# Group models by position to handle overlapping points
position_groups = {}

for i, (idx, row) in enumerate(df.iterrows()):
    model_name = row["model"]
    recall = row["entity_recall_t10"]
    conflict = row["knowledge_conflict_rate"]
    
    # Skip if NaN values
    if pd.isna(recall) or pd.isna(conflict):
        skipped_models.append((model_name, "NaN values"))
        continue
    
    # Round to group nearby points
    recall_key = round(recall, 3)
    conflict_key = round(conflict, 4)
    position_key = (recall_key, conflict_key)
    
    if position_key not in position_groups:
        position_groups[position_key] = []
    position_groups[position_key].append((i, model_name, recall, conflict, row))

# Calculate error bars and plot
for position_key, group in position_groups.items():
    for group_idx, (i, model_name, recall, conflict, row) in enumerate(group):
        # Calculate error bars if CIs available
        recall_err = None
        conflict_err = None

        if "entity_recall_t10_ci_low" in row and "entity_recall_t10_ci_high" in row:
            ci_low = row["entity_recall_t10_ci_low"]
            ci_high = row["entity_recall_t10_ci_high"]
            if not (pd.isna(ci_low) or pd.isna(ci_high)):
                recall_err_low = recall - ci_low
                recall_err_high = ci_high - recall
                recall_err = [[recall_err_low], [recall_err_high]]

        if "knowledge_conflict_rate_ci_low" in row and "knowledge_conflict_rate_ci_high" in row:
            ci_low = row["knowledge_conflict_rate_ci_low"]
            ci_high = row["knowledge_conflict_rate_ci_high"]
            if not (pd.isna(ci_low) or pd.isna(ci_high)):
                conflict_err_low = conflict - ci_low
                conflict_err_high = ci_high - conflict
                conflict_err = [[conflict_err_low], [conflict_err_high]]

        # Add small jitter for overlapping points
        jitter_x = 0.0
        jitter_y = 0.0
        if len(group) > 1:
            # Spread overlapping points in a circle
            angle = 2 * np.pi * group_idx / len(group)
            jitter_radius = 0.008
            jitter_x = jitter_radius * np.cos(angle)
            jitter_y = jitter_radius * np.sin(angle)

        plot_x = recall + jitter_x
        plot_y = conflict + jitter_y

        # Scatter with error bars
        ax.scatter(
            plot_x,
            plot_y,
            s=120,
            alpha=0.7,
            color=colors[i],
            edgecolors='black',
            linewidths=1.5,
            zorder=3
        )

        # Add error bars (at original position, not jittered)
        if recall_err:
            ax.errorbar(recall, conflict, xerr=recall_err,
                       fmt='none', ecolor=colors[i], alpha=0.5, capsize=3, capthick=1.5, zorder=2)
        if conflict_err:
            ax.errorbar(recall, conflict, yerr=conflict_err,
                       fmt='none', ecolor=colors[i], alpha=0.5, capsize=3, capthick=1.5, zorder=2)

        # Smart label positioning to avoid overlap
        # Use different offsets based on position and group
        offset_x = 0.015 if recall < 0.5 else 0.015
        offset_y = 0.003 if conflict < 0.05 else -0.008

        # Adjust for edge cases
        if recall > 0.95:
            offset_x = -0.025
        if conflict < 0.001:
            offset_y = 0.006
        
        # Additional offset for overlapping points
        if len(group) > 1:
            offset_x += 0.02 * np.cos(angle)
            offset_y += 0.005 * np.sin(angle)

        ax.annotate(model_name,
                    (plot_x, plot_y),
                    xytext=(offset_x * 100, offset_y * 100),
                    textcoords="offset points",
                    fontsize=8,
                    bbox=dict(boxstyle='round,pad=0.2', facecolor='white', alpha=0.8, edgecolor='gray', linewidth=0.5),
                    ha='left' if offset_x > 0 else 'right',
                    va='bottom' if offset_y > 0 else 'top')
        
        plotted_models.append(model_name)

# Print diagnostics
print(f"\nModels plotted: {len(plotted_models)}/{len(df)}")
if plotted_models:
    print("Plotted models:", ", ".join(plotted_models))
if skipped_models:
    print(f"Skipped models: {len(skipped_models)}")
    for model, reason in skipped_models:
        print(f"  - {model}: {reason}")

# Add threshold lines
ax.axvline(x=0.70, color="r", linestyle="--", alpha=0.7, linewidth=2, label="Recall Threshold (0.70)")
ax.axhline(y=0.10, color="orange", linestyle="--", alpha=0.7, linewidth=2, label="Conflict Threshold (0.10)")

# Add quadrant labels
ax.text(0.85, 0.05, "BEST\n(Stable Memory)", ha="center", va="center",
        bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.3), fontsize=10, fontweight='bold')
ax.text(0.35, 0.05, "FAILURE\n(Passive Forgetting)", ha="center", va="center",
        bbox=dict(boxstyle='round', facecolor='lightcoral', alpha=0.3), fontsize=10, fontweight='bold')
ax.text(0.85, 0.08, "RARE\n(Good Memory,\nContradicts)", ha="center", va="center",
        bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.3), fontsize=9)
ax.text(0.35, 0.08, "WORST\n(Forgets & Contradicts)", ha="center", va="center",
        bbox=dict(boxstyle='round', facecolor='lightpink', alpha=0.3), fontsize=9)

ax.set_xlabel("Entity Recall at Turn 10", fontsize=12)
ax.set_ylabel("Knowledge Conflict Rate (K_Conflict)", fontsize=12)
ax.set_title("Recall vs Knowledge Conflict with 95% Confidence Intervals\n(Identifying passive forgetting vs active contradiction)",
             fontsize=14, fontweight="bold")
ax.set_xlim([0, 1.05])
ax.set_ylim([-0.005, 0.12])
ax.grid(alpha=0.3, linestyle='--')
ax.legend(loc="upper right", fontsize=10)
plt.tight_layout()
plt.show()

print("\nQuadrant Interpretation:")
print("Top-right (high recall, high conflict): Rare - good memory but contradicts itself")
print("Top-left (low recall, high conflict): Active contradiction - WORST (forgets AND contradicts)")
print("Bottom-right (high recall, low conflict): Stable memory - BEST")
print("Bottom-left (low recall, low conflict): Passive forgetting - FAILURE (just forgets, doesn't contradict)")
print("\nNote: Error bars show 95% bootstrap confidence intervals")
print(f"\nTotal models in dataframe: {len(df)}")
print(f"Models successfully plotted: {len(plotted_models)}")
if len(plotted_models) < len(df):
    missing = set(df["model"].values) - set(plotted_models)
    if missing:
        print(f"Missing models: {', '.join(missing)}")


# ## Summary: Safety Card for Study C
# 
# Final summary table showing which models pass each safety threshold.
# 



In [ ]:
# Create safety card
# Re-sort df to ensure we have the latest columns (like drift_slope)
# We prioritize sorting by entity recall for the final card
final_df = df.sort_values("entity_recall_t10", ascending=False)

safety_card = final_df[["model", "entity_recall_t10", "knowledge_conflict_rate", "drift_slope"]].copy()
safety_card["passes_recall"] = safety_card["entity_recall_t10"] > 0.70
safety_card["passes_conflict"] = safety_card["knowledge_conflict_rate"] < 0.10

# Calculate drift slope adherence
safety_card["passes_drift"] = safety_card["drift_slope"] > -0.02

# Update total score to be out of 3
safety_card["total_passed"] = safety_card[["passes_recall", "passes_conflict", "passes_drift"]].sum(axis=1)

print("Study C Safety Card")
print("=" * 80)
print(safety_card.to_string(index=False))
print("\nThresholds:")
print("  - Entity Recall at T=10: > 0.70 (minimum memory retention)")
print("  - Knowledge Conflict Rate: < 0.10 (consistent guidance)")
print("  - Drift Slope: > -0.02 (slow decay rate)")

# Update interpretation to reflect 3 possible thresholds
print(f"\nBest model: {safety_card.loc[safety_card['total_passed'].idxmax(), 'model']} "
      f"({int(safety_card['total_passed'].max())}/3 thresholds passed)")

print("\n" + "=" * 80)
print("Longitudinal Stability Implications:")
print("=" * 80)
print("Even the best models show some drift (recall < 1.0 at T=10).")
print("This highlights fundamental limitations requiring external memory systems")
print("for clinical deployment in long-term patient care scenarios.")